In [8]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [9]:
# Dataset loading and preproccessing

news_test = datasets.load_files('./category', encoding='utf-8')

x_train, x_test, y_train, y_test = train_test_split(news_test.data, news_test.target, test_size=0.2)
print(len(x_test))


KeyboardInterrupt: 

In [3]:
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(x_train)
x_test_counts = count_vect.fit_transform(x_test)
x_train_counts.shape

tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)
x_test_tfidf = tfidf_transformer.fit_transform(x_test_counts)
x_train_tfidf.shape



NameError: name 'X' is not defined

In [4]:
# Neural network model 

# Parameters
learning_rate = 0.0001
epochs = 1000
display_step = 100

input_num = 18
output_num = 1

layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50


# Input Layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, input_num))
    
# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable("weights1", shape=[input_num, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable("weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable("weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Output Layer
with tf.variable_scope('output'):
    weights = tf.get_variable("weights4", shape=[layer_3_nodes, output_num], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[output_num], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

    
# The cost function of the neural network

with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

# The optimizer function

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
#Initialize session

with tf.Session() as session:

    session.run(tf.global_variables_initializer())

    for epoch in range(epochs):

        # Feed in the training data 
        session.run(optimizer, feed_dict={
                                    X: x_train_tfidf, 
                                    Y: y_train
                                })

        if epoch % display_step == 0:
            training_cost = session.run(cost, feed_dict={
                                    X: x_train_tfidf, 
                                    Y: y_train
                                })

            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(training_cost))

    print("Training finished!")

    testing_cost = session.run(cost, feed_dict={
                                    X: x_test_tfidf, 
                                    Y: y_test
                                })
    print("Mean squared error of test data: ", "{:.9f}".format(testing_cost))
    